In [79]:
import pandas as pd
import dash
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import plotly.express as px
import json
import geojson
import math

In [80]:
def load_and_preprocess_odjemalci():
    df_odjemalci = pd.read_csv("data/podatki_vrsta_odjemalca.csv", delimiter=",", encoding="windows-1250")
    df_starost_prebivalcev = pd.read_csv("data/podatki_starost_prebivalcev.csv", delimiter=",", encoding="windows-1250")

    df_map_slovenija = gpd.read_file('slovenija_map/obcine/obcine.shp')
    df_map_slovenija = df_map_slovenija.to_crs("WGS84")
    df_map_slovenija = df_map_slovenija.rename({'NAZIV': 'District'}, axis = 'columns')
    df_map_slovenija = df_map_slovenija.drop(columns = ['EID_OBCINA',  'SIFRA', 'NAZIV_DJ', 'OZNAKA_MES', 'DATUM_SYS'])
    df_map_slovenija = df_map_slovenija.sort_values('District')
    df_map_slovenija = df_map_slovenija.reset_index(drop=True)

    df_odjemalci = df_odjemalci.drop(df_odjemalci.columns[[2,3,4,5,6,7,8,9,10,11,12,13,26,27,28,29,30,31,32,33,34]], axis=1)
    df_odjemalci["Poraba"] = df_odjemalci.iloc[:,2:14].sum(axis=1)
    df_odjemalci = df_odjemalci.drop(df_odjemalci.columns[[1,2,3,4,5,6,7,8,9,10,11,12,13]], axis=1)

    df_odjemalci_skupaj = df_odjemalci[3:-3:+3]
    df_odjemalci_gospodinjstva = df_odjemalci[4:-3:+3]
    df_odjemalci_industrija = df_odjemalci[5:-3:+3]
    df_starost_prebivalcev = df_starost_prebivalcev[1::]

    df_odjemalci_skupaj = df_odjemalci_skupaj.sort_values('OBČINE')
    df_odjemalci_gospodinjstva = df_odjemalci_gospodinjstva.sort_values('OBČINE')
    df_odjemalci_industrija = df_odjemalci_industrija.sort_values('OBČINE')
    df_starost_prebivalcev = df_starost_prebivalcev.sort_values('OBČINE')

    obcine = []
    for obcina in list(df_odjemalci_skupaj["OBČINE"]):
        if "/" in obcina:
            obcina = obcina.split("/")[0]
        obcine.append(obcina)

    df_odjemalci_skupaj["OBČINE"] = obcine
    df_odjemalci_gospodinjstva["OBČINE"] = obcine
    df_odjemalci_industrija["OBČINE"] = obcine
    df_starost_prebivalcev["OBČINE"] = obcine
    df_map_slovenija["District"] = obcine

    df_odjemalci_skupaj = df_odjemalci_skupaj.reset_index(drop=True)
    df_odjemalci_gospodinjstva = df_odjemalci_gospodinjstva.reset_index(drop=True)
    df_odjemalci_industrija = df_odjemalci_industrija.reset_index(drop=True)
    df_starost_prebivalcev = df_starost_prebivalcev.reset_index(drop=True)

    df_starost_prebivalcev.drop(['2020H1 Starost - SKUPAJ', '2020H2 Starost - SKUPAJ', '2021H1 Starost - SKUPAJ'], axis=1, inplace=True)
    df_starost_prebivalcev.rename(columns={'2021H2 Starost - SKUPAJ':'POPULACIJA'}, inplace=True)

    df_odjemalci_gospodinjstva_norm = pd.DataFrame({'OBČINE': df_odjemalci_gospodinjstva['OBČINE'],
                                                    'Poraba': df_odjemalci_gospodinjstva["Poraba"]/df_starost_prebivalcev["POPULACIJA"]
                                                    })

    odjemalci_dict = {
        'skupaj': df_odjemalci_skupaj,
        'gospodinjstvo': df_odjemalci_gospodinjstva,
        'industrija': df_odjemalci_industrija,
        'gospodinjstvo_norm': df_odjemalci_gospodinjstva_norm,
        'df_map': df_map_slovenija
    }

    return odjemalci_dict

odjemalci_dict = load_and_preprocess_odjemalci()

In [85]:
list(odjemalci_dict['skupaj']['OBČINE'])

['Ajdovščina',
 'Ankaran',
 'Apače',
 'Beltinci',
 'Benedikt',
 'Bistrica ob Sotli',
 'Bled',
 'Bloke',
 'Bohinj',
 'Borovnica',
 'Bovec',
 'Braslovče',
 'Brda',
 'Brezovica',
 'Brežice',
 'Cankova',
 'Celje',
 'Cerklje na Gorenjskem',
 'Cerknica',
 'Cerkno',
 'Cerkvenjak',
 'Cirkulane',
 'Destrnik',
 'Divača',
 'Dobje',
 'Dobrepolje',
 'Dobrna',
 'Dobrova - Polhov Gradec',
 'Dobrovnik',
 'Dol pri Ljubljani',
 'Dolenjske Toplice',
 'Domžale',
 'Dornava',
 'Dravograd',
 'Duplek',
 'Gorenja vas - Poljane',
 'Gorišnica',
 'Gorje',
 'Gornja Radgona',
 'Gornji Grad',
 'Gornji Petrovci',
 'Grad',
 'Grosuplje',
 'Hajdina',
 'Hodoš',
 'Horjul',
 'Hoče - Slivnica',
 'Hrastnik',
 'Hrpelje - Kozina',
 'Idrija',
 'Ig',
 'Ilirska Bistrica',
 'Ivančna Gorica',
 'Izola',
 'Jesenice',
 'Jezersko',
 'Juršinci',
 'Kamnik',
 'Kanal',
 'Kidričevo',
 'Kobarid',
 'Kobilje',
 'Komen',
 'Komenda',
 'Koper',
 'Kostanjevica na Krki',
 'Kostel',
 'Kozje',
 'Kočevje',
 'Kranj',
 'Kranjska Gora',
 'Križevci',
 'Kr

In [ ]:
def load_and_preprocess_dejavnosti():
    df_dejavnosti = pd.read_csv("data/podatki_dejavnosti.csv", delimiter=",", encoding="windows-1250")
    dejavnosti_dict = {}

    for i in range(23, 45):
        df = df_dejavnosti[i:-i:+i]
        dejavnosti_dict[str(df_dejavnosti.iloc[i]['SKD DEJAVNOST'])[2:]] = df

    return dejavnosti_dict

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

l = [i['properties']['NAME'] for i in counties["features"]]
l.sort()

for i in l: print(i)

In [56]:
import pandas as pd

def izrisi_dejavnosti_obcine():
    df = pd.read_csv("data/podatki_dejavnosti.csv", delimiter=",", encoding="windows-1250")
    df = df.loc[(df["OBČINE"] != "SLOVENIJA") & (df["OBČINE"] != "Neznano") & (df['SKD DEJAVNOST'] != 'SKD Dejavnost - SKUPAJ') & (df['SKD DEJAVNOST'] != 'X Neznano')]
    dejavnosti = list(df["SKD DEJAVNOST"].unique())
    obcine = list(df['OBČINE'].unique())
    df = df.rename(columns={'2021 (začasni podatki)': 'Poraba'})
    dejavnosti_dict = {}
    
    for dejavnost in dejavnosti:
        df1 = df.loc[df["SKD DEJAVNOST"] == dejavnost].drop(columns=("SKD DEJAVNOST"))
        obcine = []
        for obcina in list(df1["OBČINE"]):
            if "/" in obcina:
                obcina = obcina.split("/")[0]
            obcine.append(obcina)
        df1["OBČINE"] = obcine
        if df1["Poraba"].sum() != 0:
            dejavnosti_dict[" ".join(dejavnost.split(" ")[1:])] = df1
    return dejavnosti_dict

dejavnosti_dict = izrisi_dejavnosti_obcine()

In [76]:
dejavnosti_dict

{'Kmetijstvo in lov, gozdarstvo, ribištvo':           OBČINE  Poraba
 24    Ajdovščina   75744
 47       Ankaran       0
 70         Apače   24105
 93      Beltinci  593013
 116     Benedikt       0
 ...          ...     ...
 4785   Železniki   28967
 4808      Žetale    2415
 4831        Žiri    8345
 4854   Žirovnica    2075
 4877   Žužemberk   25068
 
 [212 rows x 2 columns],
 'Rudarstvo':           OBČINE  Poraba
 25    Ajdovščina       0
 48       Ankaran       0
 71         Apače       0
 94      Beltinci       0
 117     Benedikt       0
 ...          ...     ...
 4786   Železniki       0
 4809      Žetale       0
 4832        Žiri       0
 4855   Žirovnica       0
 4878   Žužemberk       0
 
 [212 rows x 2 columns],
 'Predelovalne dejavnosti':           OBČINE    Poraba
 26    Ajdovščina  47685321
 49       Ankaran    216838
 72         Apače   1038304
 95      Beltinci   3678594
 118     Benedikt    412055
 ...          ...       ...
 4787   Železniki  25396218
 4810      Žeta

In [74]:
import numpy as np

def calculate_sums_dejavnosti(list_dejavnosti, dejavnosti_d):
    seznami = []
    for dejavnost in list_dejavnosti:
        seznami.append(list(dejavnosti_d[dejavnost]["Poraba"].values))
    return list(sum(map(np.array, np.array(seznami))))

In [75]:
l = ['Kmetijstvo in lov, gozdarstvo, ribištvo',
     'Rudarstvo', 'Oskrba z električno energijo, plinom in paro', 
     'Predelovalne dejavnosti', 
     'Oskrba z vodo, ravnanje z odplakami in odpadki, saniranje okolja']

calculate_sums_dejavnosti(l, dejavnosti_dict)

[49711952,
 2381501,
 1623883,
 6223483,
 521716,
 453408,
 8050540,
 3055959,
 9232214,
 10150028,
 8554800,
 1990546,
 1955504,
 2462582,
 14213716,
 377638,
 167549884,
 10169391,
 27118414,
 32312483,
 82080,
 2361782,
 611729,
 62429507,
 44071138,
 885953,
 9898673,
 40776,
 3229896,
 524529,
 1522611,
 880086,
 3929517,
 1293366,
 188033632,
 535240,
 13758068,
 457123,
 14334503,
 2118462,
 856055,
 19868036,
 1496675,
 899471,
 118795,
 39629287,
 4951237,
 33415509,
 193319,
 7490107,
 90457419,
 7007611,
 48296537,
 4833611,
 86585663,
 29572514,
 7860416,
 16302360,
 34626,
 198078,
 174361652,
 141273709,
 9726893,
 7591154,
 4809,
 36128116,
 4596102,
 21612748,
 54526076,
 4070103,
 685704,
 3427865,
 154375889,
 1484625,
 6531611,
 113601529,
 1056330,
 113658,
 27421734,
 17227762,
 73068018,
 41060601,
 496280699,
 20145876,
 26071000,
 980106,
 35242347,
 21068107,
 1765866,
 2433765,
 1280371,
 1682270,
 672571,
 713724,
 136278993,
 7858201,
 71652790,
 44708876,
 